In [1]:
import pandas as pd

In [2]:
result_file = "test_result_w_succ_diff_nodes_lstm_2025_04_03_15_38.csv"

In [3]:
EPS = 1

In [4]:
df = pd.read_csv(result_file)

df = df.rename(columns={"Dataset": "Graph", "Actual": "Rank"})

df.head()

,Graph,Rank,Predicted
0,star_graph_n7,0.0,0.034314
1,star_graph_n7,0.0,0.000381
2,star_graph_n7,0.0,0.012208
3,star_graph_n7,0.0,0.216689
4,star_graph_n7,0.0,0.080881


In [5]:
df['Difference'] = abs(df['Rank']-df['Predicted'])
df['Difference_sq'] = df['Difference']**2
# df.head()

In [6]:
df['EPS'] = df['Rank'].apply(lambda x: EPS if x==0.0 else 0.0)
# df

In [7]:
df['RE'] = df['Difference']/(df['Rank']+df['EPS'])
# df.head()

In [8]:
df

,Graph,Rank,Predicted,Difference,Difference_sq,EPS,RE
0,star_graph_n7,0.0,0.034314,0.034314,1.177443e-03,1.0,0.034314
1,star_graph_n7,0.0,0.000381,0.000381,1.452594e-07,1.0,0.000381
2,star_graph_n7,0.0,0.012208,0.012208,1.490297e-04,1.0,0.012208
3,star_graph_n7,0.0,0.216689,0.216689,4.695407e-02,1.0,0.216689
4,star_graph_n7,0.0,0.080881,0.080881,6.541747e-03,1.0,0.080881
...,...,...,...,...,...,...,...
5275,graph_powerlaw_cluster_graph_n7,2.0,2.669063,0.669063,4.476451e-01,0.0,0.334531
5276,graph_powerlaw_cluster_graph_n7,2.0,1.983100,0.016900,2.856202e-04,0.0,0.008450
5277,graph_powerlaw_cluster_graph_n7,2.0,2.017103,0.017103,2.925111e-04,0.0,0.008551
5278,graph_powerlaw_cluster_graph_n7,1.0,1.018844,0.018844,3.551146e-04,0.0,0.018844


In [9]:
df['RE'].sum()/len(df)

0.05503705658784696

In [10]:
def group_by_table(df, groupby):
    values_count = df[groupby].value_counts().reset_index()
    values_count = values_count.rename(columns={"count": "Count"})
    re_sum = df.groupby(groupby)["RE"].sum().reset_index()
    mre_error = df.groupby(groupby)["RE"].mean().reset_index()
    mre_error = mre_error.rename(columns={"RE": "MRE"})
    mse_error = df.groupby(groupby)["Difference_sq"].mean().reset_index()
    mse_error = mse_error.rename(columns={"Difference_sq": "MSE"})

    return values_count, re_sum, mre_error, mse_error

# Group by Rank

In [11]:
groupby = ['Rank']

values_count, re_sum, mre_error, mse_error = group_by_table(df, groupby)

df_result = pd.merge(values_count, re_sum).sort_values(by=['Count', 'RE'], ascending=[False, False])
df_result = pd.merge(df_result, mre_error)
df_result = pd.merge(df_result, mse_error)
df_result.to_csv("analysis_results/test_analysis_grp_by_rank.csv", index=False)
df_result


,Rank,Count,RE,MRE,MSE
0,3.0,1715,100.088123,0.058360,0.050713
1,2.0,1615,115.280364,0.071381,0.043970
2,1.0,928,40.456399,0.043595,0.004329
3,4.0,681,28.768254,0.042244,0.046468
4,0.0,267,1.046333,0.003919,0.000384
5,5.0,74,4.956187,0.066976,0.175390


# Group by Dataset

In [12]:
groupby = ['Graph']

values_count, re_sum, mre_error, mse_error = group_by_table(df, groupby)

df_result = pd.merge(values_count, re_sum).sort_values(by=['Count', 'RE'], ascending=[False, False])
df_result = pd.merge(df_result, mre_error)
df_result = pd.merge(df_result, mse_error)
df_result.to_csv("analysis_results/test_analysis_grp_by_dataset.csv", index=False)
df_result


,Graph,Count,RE,MRE,MSE
0,graph_random_regular_graph_n7_d4,3907,203.690048,0.052135,0.036179
1,graph_powerlaw_cluster_graph_n7,1350,85.425365,0.063278,0.047841
2,star_graph_n7,23,1.480246,0.064359,0.034330


# Group by Dataset W/O Rank 0

In [13]:
groupby = ['Graph']

df_wo_0 = df[df['Rank']!=0]
values_count, re_sum, mre_error, mse_error = group_by_table(df_wo_0, groupby)

df_result = pd.merge(values_count, re_sum).sort_values(by=['Count', 'RE'], ascending=[False, False])
df_result = pd.merge(df_result, mre_error)
df_result = pd.merge(df_result, mse_error)
df_result.to_csv("analysis_results/test_analysis_grp_by_dataset_wo_rank0.csv", index=False)
df_result

,Graph,Count,RE,MRE,MSE
0,graph_random_regular_graph_n7_d4,3766,203.613950,0.054066,0.037533
1,graph_powerlaw_cluster_graph_n7,1240,85.357707,0.068837,0.052083
2,star_graph_n7,7,0.577670,0.082524,0.098601


# Group by Dataset and Rank

In [14]:
groupby = ['Graph', 'Rank']

values_count, re_sum, mre_error, mse_error = group_by_table(df, groupby)

df_result = pd.merge(values_count, re_sum).sort_values(by=['Graph', 'Count'], ascending=[True, False])
df_result = pd.merge(df_result, mre_error)
df_result = pd.merge(df_result, mse_error)
df_result.to_csv("analysis_results/test_analysis_grp_by_dataset_rank.csv", index=False)
df_result

,Graph,Rank,Count,RE,MRE,MSE
0,graph_powerlaw_cluster_graph_n7,2.0,406,39.931992,0.098355,0.080132
1,graph_powerlaw_cluster_graph_n7,1.0,357,17.427614,0.048817,0.006181
2,graph_powerlaw_cluster_graph_n7,3.0,343,23.090160,0.067318,0.070012
3,graph_powerlaw_cluster_graph_n7,4.0,115,3.864109,0.033601,0.032185
4,graph_powerlaw_cluster_graph_n7,0.0,110,0.067658,0.000615,0.000014
5,graph_powerlaw_cluster_graph_n7,5.0,19,1.043832,0.054939,0.111957
6,graph_random_regular_graph_n7_d4,3.0,1370,76.796202,0.056056,0.045738
7,graph_random_regular_graph_n7_d4,2.0,1208,75.220037,0.062268,0.031797
8,graph_random_regular_graph_n7_d4,1.0,570,22.979952,0.040316,0.003172
9,graph_random_regular_graph_n7_d4,4.0,564,24.775550,0.043928,0.049188
